In [1]:
from datasets import load_dataset
ashaar = load_dataset("arbml/ashaar")

Found cached dataset parquet (/home/alyafey22/.cache/huggingface/datasets/arbml___parquet/MagedSaeed--ashaar-719bb58a76ea0092/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
# ashaar['train'] = ashaar['train'].select(range(10000))

In [2]:
def join_meters(sample):
    meter = sample['poem meter']
    remove_texts = ['مجزوء', 'مشطور', 'أحذ', 'مخلع', 'مربع', 'التفعيلة', 'منهوك', 'بحر', 'تفعيلة' , 'المقطوع']
    if meter:
        for removed in remove_texts:
            sample['poem meter'] = sample['poem meter'].replace(removed, "").strip()
    return sample

In [3]:
ashaar['train'] = ashaar['train'].map(join_meters)

Loading cached processed dataset at /home/alyafey22/.cache/huggingface/datasets/arbml___parquet/MagedSaeed--ashaar-719bb58a76ea0092/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-bee65d4186575ff7.arrow


In [4]:
meters = ['الكامل', 'المتدارك', 'الوافر', 'الرمل', 'السريع', 'الهزج', 'المقتضب', 'الطويل', 'المتقارب', 'المتدارك', 'المضارع', 'المديد', 'البسيط', 'الرجز', 'المنسرح', 'المجتث', 'الخفيف']

In [5]:
ashaar = ashaar.filter(lambda example: example["poem meter"] in meters)

Loading cached processed dataset at /home/alyafey22/.cache/huggingface/datasets/arbml___parquet/MagedSaeed--ashaar-719bb58a76ea0092/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-cf1ee173cd1b4b0f.arrow


In [6]:
meters= {}
unique_meters = set(x for x in ashaar['train']['poem meter'] if x is not None)
meter_tokens = {meter:f'<|meter_{i}|>' for i, meter in enumerate(unique_meters)}
print(meter_tokens)

{'المديد': '<|meter_0|>', 'السريع': '<|meter_1|>', 'الرجز': '<|meter_2|>', 'الكامل': '<|meter_3|>', 'المتقارب': '<|meter_4|>', 'المجتث': '<|meter_5|>', 'المضارع': '<|meter_6|>', 'الخفيف': '<|meter_7|>', 'الرمل': '<|meter_8|>', 'البسيط': '<|meter_9|>', 'الهزج': '<|meter_10|>', 'الطويل': '<|meter_11|>', 'المنسرح': '<|meter_12|>', 'المقتضب': '<|meter_13|>', 'المتدارك': '<|meter_14|>', 'الوافر': '<|meter_15|>'}


In [7]:
ST_POEM_TOKEN = '<|psep|>'
ED_POEM_TOKEN = '</|psep|>'
VERSE_TOKEN = '<|vsep|>'
ST_BAYT_TOKEN= '<|bsep|>'
ED_BAYT_TOKEN= '</|bsep|>'
PAD_TOKEN = '<|pad|>'
EOT_TOKEN = '<|endoftext|>'

In [64]:
def process_verse(sample):
    chars = 'ابتثجحخدذرزسشصضطظعغفقكلمنهويىئءأؤة ى'
    diacs = 'ْ~ًٌٍَُِّ'
    map_chars = {'ک':'ك', 'ﺑ':'ب', 'ٹ':'ث', 'ی':'ى'}
    out = ''
    for char in sample:
        if char in chars+diacs:
            out += char
        elif char in map_chars:
            out += map_chars[char]
    return out.strip()

def join_verses(sample):

    verses = sample['poem verses']
    meter = sample['poem meter']
  
    poem = ''.join([ f'{ST_BAYT_TOKEN} '+process_verse(verses[i]) +f' {VERSE_TOKEN} '+ process_verse(verses[i+1])+ f' {ED_BAYT_TOKEN} ' for i in range(0, len(verses)-2, 2)])
    
    if meter:
        meter_token = meter_tokens[meter]
    else:
        meter_token = ''
    text = f"{meter_tokens[meter]} {ST_POEM_TOKEN} {poem.strip()} {ED_POEM_TOKEN}"

    return {'text':text}

In [65]:
ashaar['train'] = ashaar['train'].map(join_verses)

Map:   0%|          | 0/144188 [00:00<?, ? examples/s]

In [66]:
ashaar['train'][0]

{'poem title': 'أصبح الملك للذي فطر الخلق',
 'poem meter': 'الخفيف',
 'poem verses': ['أَصبَحَ المُلك لِلَّذي فَطر الخَل',
  'قَ بِتَقديرٍ للعَزيز العَليمِ',
  'غافر الذَنب للمسيءِ بِعَفوٍ',
  'قابل التَوب ذي العَطاء العَميمِ',
  'مُرسل المُصطَفى البَشير إِلَينا',
  'رَحمة مِنهُ بِالكَلام القَديمِ',
  'رَبَنا رَبّنا إِلَيكَ أَنينا',
  'فَأَجرنا مِن حَر نار الجَحيمِ',
  'وَاكفِنا شَرّ ما نَخاف بِلُطفٍ',
  'يا عَظيماً يَرجى لِكُل عَظيمِ',
  'وَتَقبل أَعمالَنا وَاعفُ عَنا',
  'وَأَنلنا دُخول دار النَعيمِ',
  'بِنَبي بَعثَتهُ فَهَدانا',
  'لِصِراط مِن الهُدى مُستَقيمِ',
  'وَبِمَن نَحنُ في حِماهُ مَدى الدَهر',
  'أَخيهِ يَحيى الحصور الكَريمِ',
  'أَدرك أَدرك قَوماً أَتوا بافتقار',
  'وَاِنكِسار وَمَدمَع مَسجومِ',
  'شَهدت أَرواحَهُم أَنكَ اللَهُ',
  'وَجاءوا بِكُل قَلبٍ سَليم'],
 'poem theme': 'قصيدة دينية',
 'poem url': 'https://www.aldiwan.net/poem16182.html',
 'poet name': 'الامير منجك باشا',
 'poet description': 'منجك بن محمد بن منجك بن ابي بكر بن عبد القادر بن ابراهيم بن منجك اليوسفي 

In [67]:
batch_size = 1000

def batch_iterator():
    for i in range(0, len(ashaar['train']), batch_size):
        yield ashaar['train'][i : i + batch_size]["text"]

In [68]:
full_dataset = ""
for text in ashaar['train']["text"]:
    full_dataset += text
char_cnt =len(set(list(full_dataset)))

In [69]:
print(char_cnt)

67


In [70]:
print(set(list(full_dataset)))

{'1', '7', 'ة', 'س', 'ح', '<', 'p', 'ً', 'ف', 's', 'و', 'ُ', 'ك', 'ٍ', 'r', '5', 'ص', 'ط', 'ق', '9', 'د', 'ي', 'ض', '_', 'خ', 'ء', 'ث', '0', '~', 'ى', 'e', 'm', ' ', 'َ', 'ٌ', 'ر', 'v', 'ج', 'م', 'ا', 'ن', 'ه', 'ؤ', 'ظ', 'b', '2', '/', 'ت', '8', 'ّ', 't', 'ل', '3', 'ِ', 'ب', 'ذ', 'ز', 'أ', 'ع', 'غ', 'ش', 'ئ', 'ْ', '4', '6', '>', '|'}


In [71]:
print(list(meter_tokens.values()))

['<|meter_0|>', '<|meter_1|>', '<|meter_2|>', '<|meter_3|>', '<|meter_4|>', '<|meter_5|>', '<|meter_6|>', '<|meter_7|>', '<|meter_8|>', '<|meter_9|>', '<|meter_10|>', '<|meter_11|>', '<|meter_12|>', '<|meter_13|>', '<|meter_14|>', '<|meter_15|>']


In [78]:
extra_tokens = [f'<|res_{i}|>' for i in range(10)]
print(extra_tokens)

['<|res_0|>', '<|res_1|>', '<|res_2|>', '<|res_3|>', '<|res_4|>', '<|res_5|>', '<|res_6|>', '<|res_7|>', '<|res_8|>', '<|res_9|>']


In [79]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer
tokenizer = Tokenizer(models.BPE())
#This pre-tokenizer takes care of replacing all bytes of the given string with a corresponding representation, as well as splitting into words.
tokenizer.pre_tokenizer = pre_tokenizers.UnicodeScripts() #add space whether to add space at the first word
special_tokens = [EOT_TOKEN, ST_POEM_TOKEN, ED_POEM_TOKEN, VERSE_TOKEN, ST_BAYT_TOKEN, ED_BAYT_TOKEN, PAD_TOKEN]+list(meter_tokens.values())+extra_tokens
trainer = trainers.BpeTrainer(vocab_size=char_cnt+len(special_tokens), special_tokens=special_tokens)
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)
tokenizer.pad_token = PAD_TOKEN
print(tokenizer.get_vocab_size())




100


In [80]:
from transformers import GPT2TokenizerFast, AutoTokenizer
gpt_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

In [81]:
print([(gpt_tokenizer.tokenize(sp)[0], sp) for sp in special_tokens])

[('<|endoftext|>', '<|endoftext|>'), ('<|psep|>', '<|psep|>'), ('</|psep|>', '</|psep|>'), ('<|vsep|>', '<|vsep|>'), ('<|bsep|>', '<|bsep|>'), ('</|bsep|>', '</|bsep|>'), ('<|pad|>', '<|pad|>'), ('<|meter_0|>', '<|meter_0|>'), ('<|meter_1|>', '<|meter_1|>'), ('<|meter_2|>', '<|meter_2|>'), ('<|meter_3|>', '<|meter_3|>'), ('<|meter_4|>', '<|meter_4|>'), ('<|meter_5|>', '<|meter_5|>'), ('<|meter_6|>', '<|meter_6|>'), ('<|meter_7|>', '<|meter_7|>'), ('<|meter_8|>', '<|meter_8|>'), ('<|meter_9|>', '<|meter_9|>'), ('<|meter_10|>', '<|meter_10|>'), ('<|meter_11|>', '<|meter_11|>'), ('<|meter_12|>', '<|meter_12|>'), ('<|meter_13|>', '<|meter_13|>'), ('<|meter_14|>', '<|meter_14|>'), ('<|meter_15|>', '<|meter_15|>'), ('<|res_0|>', '<|res_0|>'), ('<|res_1|>', '<|res_1|>'), ('<|res_2|>', '<|res_2|>'), ('<|res_3|>', '<|res_3|>'), ('<|res_4|>', '<|res_4|>'), ('<|res_5|>', '<|res_5|>'), ('<|res_6|>', '<|res_6|>'), ('<|res_7|>', '<|res_7|>'), ('<|res_8|>', '<|res_8|>'), ('<|res_9|>', '<|res_9|>')]


In [83]:
len(gpt_tokenizer.get_vocab())

100

In [85]:
ashaar.push_to_hub('ashaar_dataset')

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/73 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/73 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/2 [00:00<?, ?it/s]

In [86]:
gpt_tokenizer.push_to_hub('ashaar_tokenizer')

CommitInfo(commit_url='https://huggingface.co/Zaid/ashaar_tokenizer/commit/5f50cb4918e5507094340d52428c375a244766aa', commit_message='Upload tokenizer', commit_description='', oid='5f50cb4918e5507094340d52428c375a244766aa', pr_url=None, pr_revision=None, pr_num=None)

In [87]:
from transformers import GPT2TokenizerFast, AutoTokenizer

gpt_tokenizer = GPT2TokenizerFast.from_pretrained('Zaid/ashaar_tokenizer')

In [89]:
print(gpt_tokenizer.get_vocab())

{'ظ': 79, 'م': 86, 'ي': 91, 'ض': 77, '<|res_2|>': 25, 't': 54, 'ْ': 99, '_': 47, '<|endoftext|>': 0, '1': 36, '6': 41, '>': 46, 'ر': 72, '<|meter_9|>': 16, '<|meter_12|>': 19, '~': 57, 'ث': 66, 'ٌ': 93, '<|meter_0|>': 7, '8': 43, 'p': 51, '</|bsep|>': 5, '<|meter_5|>': 12, 'ن': 87, '<|psep|>': 1, '<|vsep|>': 3, '2': 37, 'ى': 90, 'ب': 63, '<|meter_10|>': 17, 'خ': 69, 'ف': 82, '9': 44, '<|res_1|>': 24, '<|meter_4|>': 11, '<|res_7|>': 30, '|': 56, 'ٍ': 94, 'ّ': 98, '<|res_5|>': 28, 'غ': 81, '7': 42, '<|meter_11|>': 18, 'e': 49, 'ؤ': 60, '<|meter_13|>': 20, '<|meter_6|>': 13, '4': 39, '<|meter_15|>': 22, '<|res_3|>': 26, '<|pad|>': 6, '<|meter_14|>': 21, '<|meter_8|>': 15, 'ئ': 61, 'ة': 64, 'ح': 68, 'ص': 76, 'س': 74, '<': 45, 'm': 50, 'ش': 75, '<|res_8|>': 31, 'ل': 85, 'ً': 92, '<|bsep|>': 4, '<|res_6|>': 29, 's': 53, 'v': 55, 'r': 52, '<|res_4|>': 27, 'ق': 83, 'ع': 80, 'َ': 95, 'ُ': 96, '<|meter_1|>': 8, 'ِ': 97, 'د': 70, 'ط': 78, 'ء': 58, 'ك': 84, '<|meter_2|>': 9, 'ذ': 71, '/': 34, '<|m

In [90]:
chars = 'ابتثجحخدذرزسشصضطظعغفقكلمنهويىئءأؤة ى'
diacs = 'ْ~ًٌٍَُِّ'
en_chars = 'abcdefghijklmnopqrstuvwxyz/'
for vocab in gpt_tokenizer.get_vocab():
    if '<' in vocab and '>' in vocab:
#         print(vocab)
        continue
    elif vocab in chars +diacs:
        continue
    elif vocab in en_chars:
        print(vocab)
    else:
#         print(vocab)
        pass

t
p
e
m
s
v
r
/
b


In [91]:
print(gpt_tokenizer.get_vocab())

{'ظ': 79, 'م': 86, 'ي': 91, 'ض': 77, '<|res_2|>': 25, 't': 54, 'ْ': 99, '_': 47, '<|endoftext|>': 0, '1': 36, '6': 41, '>': 46, 'ر': 72, '<|meter_9|>': 16, '<|meter_12|>': 19, '~': 57, 'ث': 66, 'ٌ': 93, '<|meter_0|>': 7, '8': 43, 'p': 51, '</|bsep|>': 5, '<|meter_5|>': 12, 'ن': 87, '<|psep|>': 1, '<|vsep|>': 3, '2': 37, 'ى': 90, 'ب': 63, '<|meter_10|>': 17, 'خ': 69, 'ف': 82, '9': 44, '<|res_1|>': 24, '<|meter_4|>': 11, '<|res_7|>': 30, '|': 56, 'ٍ': 94, 'ّ': 98, '<|res_5|>': 28, 'غ': 81, '7': 42, '<|meter_11|>': 18, 'e': 49, 'ؤ': 60, '<|meter_13|>': 20, '<|meter_6|>': 13, '4': 39, '<|meter_15|>': 22, '<|res_3|>': 26, '<|pad|>': 6, '<|meter_14|>': 21, '<|meter_8|>': 15, 'ئ': 61, 'ة': 64, 'ح': 68, 'ص': 76, 'س': 74, '<': 45, 'm': 50, 'ش': 75, '<|res_8|>': 31, 'ل': 85, 'ً': 92, '<|bsep|>': 4, '<|res_6|>': 29, 's': 53, 'v': 55, 'r': 52, '<|res_4|>': 27, 'ق': 83, 'ع': 80, 'َ': 95, 'ُ': 96, '<|meter_1|>': 8, 'ِ': 97, 'د': 70, 'ط': 78, 'ء': 58, 'ك': 84, '<|meter_2|>': 9, 'ذ': 71, '/': 34, '<|m